In [ ]:
import graph_tool.all as gt                       # Biblioteca para GRAFO
import csv  
import math

In [ ]:
g = gt.Graph() 
g.set_directed(False)                             # criação do objeto
v_name = g.new_vertex_property("string")          # referenciação da lista v_name com uma nova propriedade (label) criada para o vértice - tipo string  
v_pos  = g.new_vertex_property("vector<double>")
e_distance = g.new_edge_property("string") 
#Criação dos vértices no grafo à partir do arquivo .csv de vértices
f_network = open("vertices.csv", 'r', encoding='utf-8')
reader_network = csv.reader(f_network, delimiter=",")

vposX = []
vposY = []
for vertice in reader_network:
    v = g.add_vertex()
    v_name[v] = str(vertice[1]) # Adicionando nome do vertice
    vposX.append(vertice[5]) # Adicionando posição X
    vposY.append(vertice[6]) # Adicionando posição Y
f_network.close()

#Criação das arestas no grafo à partir do arquivo .cvs de arestas
f_network = open("arestas.csv", 'r', encoding='utf-8')
reader_network = csv.reader(f_network, delimiter=",")
for edge in reader_network:
    e = g.add_edge(int(edge[0]), int(edge[1]))
    e_distance[e] = str(edge[3])
    
f_network.close()

#Preenchendo as posições X e Y dos vértices
for v in g.vertices():
    v_pos[v] = (vposX[int(v)],vposY[int(v)])

In [ ]:
class VisitorExample(gt.DFSVisitor):                                            # É um objeto visitante que é chamado nos pontos de evento dentro do algoritmo dfs_search()
    def __init__(self, name, time, name_time, v_color, dist, pred, e_color, e_action, e_ord): 
        self.name = name
        self.time = time
        self.name_time = name_time
        self.fill_color = v_color
        self.dist = dist
        self.pred = pred
        self.color = e_color
        self.e_action = e_action
        self.e_ord = e_ord
        self.e_count = 0
        self.last_time = 0
        
    def discover_vertex(self, u):                                               # Invocado quando um vértice é encontrado pela primeira vez.
        self.name[u] = v_name[u]
        self.time[u] = self.last_time
        self.last_time += 1        
        self.name_time[u] = str(self.name[u]) + "(" + str(self.time[u]) + ")"
        #print("-->", self.name[u], "foi encontrado e entrou na FILA") 
        self.fill_color[u] = "white"

   # def examine_vertex(self, u):                                                # Invocado em um vértice à medida que é retirado da fila. 
  #      print(self.name[u], "saiu da FILA e está sendo analisado (expandido)...") 

    def tree_edge(self, e):                                                     # Invocado em cada aresta à medida que se torna um 
        self.pred[e.target()] = int(e.source())                                 # membro das arestas que formam a árvore de pesquisa.
        self.dist[e.target()] = self.dist[e.source()] + 1
        e = g_dfs.add_edge(int(e.source()), int(e.target()))
        self.color[e] = "gray"
        #self.e_action[e] = e_action[g.edge(int(e.source()), int(e.target()))]
        self.e_count += 1
        self.e_ord[e] = self.e_count

   # def finish_vertex(self, u):
   #     print("Todos os vértices adjacentes à", self.name[u], "foram descobertos!") 



In [ ]:
index_Destiny = 0
index_Source = 0
for index_Destiny in range(0,61,1):
    for index_Source in range(0,61,1):
        if index_Source == index_Destiny:
            continue
        g_dfs = gt.Graph()                                    # criação do objeto para busca em Largura
        v_name_dfs = g_dfs.new_vertex_property("string")      # referenciação da lista v_name com uma nova propriedade (label) criada para o vértice - tipo string 
        e_ord = g_dfs.new_edge_property("int")                # referenciação da lista e_ord com uma nova propriedade criada para a ordem de expansão - tipo int
        #####   Iterator ######
        f_network = open("vertices.csv", 'r', encoding='utf-8')
        reader_network = csv.reader(f_network, delimiter=",")
        for vertice in reader_network:
            v = g_dfs.add_vertex()
            v_name_dfs[v] = str(vertice[1])
        f_network.close()

        ord = 1
        for edge in gt.dfs_iterator(g, g.vertex(index_Source)): 
           e = g_dfs.add_edge(int(edge.source()), int(edge.target()))
           e_ord[e] = ord
           ord += 1
        ##### Busca em largura - início #####
        g_dfs = gt.Graph()                                      # criação do objeto para busca em Profundidade
        dfsv_name       = g_dfs.new_vertex_property("string")      # referenciação da lista v_name_dfs com uma nova propriedade do vértice para o nome - tipo string 
        dfsv_time       = g_dfs.new_vertex_property("int")         # referenciação da lista v_time com uma nova propriedade do vértice para a ordem de expansão - tipo int
        dfsv_name_time  = g_dfs.new_vertex_property("string")      # referenciação da lista v_name_time com uma nova propriedade do vértice para o nome e ordem de expansão - tipo string
        dfsv_color      = g_dfs.new_vertex_property("string")      # referenciação da lista v_color com uma nova propriedade do vértice para a cor - tipo string  
        dfsv_dist       = g_dfs.new_vertex_property("int")         # referenciação da lista v_dist como uma propriedade do vértice criada para a distância da raiz
        dfsv_pred       = g_dfs.new_vertex_property("int64_t")     # referenciação da lista v_pred como uma propriedade do vértice para referenciar o predecessor (pai)
        dfse_color      = g_dfs.new_edge_property("string")        # referenciação da lista e_color com uma nova propriedade da aresta para a cor - tipo string  
        dfse_action     = g_dfs.new_edge_property("string")        # referenciação da lista e_action_dfs com uma nova propriedade da aresta para a ação - tipo string
        dfse_ord        = g_dfs.new_edge_property("string") 
        index_raiz = index_Source
   
        gt.dfs_search(g, g.vertex(index_raiz), VisitorExample(dfsv_name, dfsv_time, dfsv_name_time, dfsv_color, dfsv_dist, dfsv_pred, dfse_color, dfse_action, dfse_ord))
            
        index = index_Destiny  # Localizando o índice do Estado a ser encontrado
        path = []                                       # array do caminho
        id_caminho = []

        path.insert(0,dfsv_name[index])  # inserções sendo realizadas no início
        dfsv_color[index] = "green"
        id_caminho.insert(0, index)
        while index != index_raiz:
            e = g_dfs.edge(dfsv_pred[index], index)
            dfse_color[e] = "red"
            index = dfsv_pred[index]
            path.insert(0,dfsv_name[index])
            id_caminho.insert(0, index)
            dfsv_color[index] = "#729fcf"
            dfsv_color[index_raiz] = "#729fcf"; 
        
        print(list(id_caminho))  
        

        path = [str(id) for id in id_caminho]
        path = ",".join(path)
        _file = open(f"table_q_profundidade1/table_q_{index_Destiny}.csv", "a", encoding="utf-8")  
        _file.write(f"{path}\n")
        _file.close()
                

            
        
        